# File Search in the Responses API

Working with RAG pipelines can feel complex,  Following are setup task: 
- You must parse the file content using appropriate parser
- Choose a chunking strategy, and split the content,
- Upload those chunks somewhere,
- Generate embeddings for each chunk, and finally store those embeddings in a vector database.

Next retrieving relevant information involves its own sequence of operations like
- finalising the search algorithm
- Hyperparameters
- Filter contitions

This is where file search — a hosted tool you can use in the Responses API — comes in. 
# 🔍 What is File Search?

File Search is a built-in capability that lets models look into your uploaded documents before generating an answer. Instead of relying only on the model’s internal knowledge, you can expand its context by giving it access to your own files. It supports both semantic (meaning-based) and keyword search, ensuring that the model retrieves the most relevant information from your knowledge base. File search simplifies this entire process. It’s a hosted tool available through the Responses API that lets you query your knowledge base directly and receive responses grounded in retrieved information — without manually orchestrating the retrieval steps.

In this notebook, we’ll show how to upload PDFs into an OpenAI-managed vector store and use file search to automatically pull the right context when answering questions. To illustrate this, we’ll begin by creating a small set of queries derived from PDFs extracted from OpenAI’s news articles.



In [1]:
#!pip install PyPDF2 pandas tqdm openai -q

In [2]:
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import concurrent
import PyPDF2
import os
import pandas as pd
import base64


In [3]:
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv()

True

## Creating Vector Store with our PDFs

In [4]:

client = OpenAI()
dir_pdfs = './data/openai_blog_pdfs' # have those PDFs stored locally here
pdf_files = [os.path.join(dir_pdfs, f) for f in os.listdir(dir_pdfs)]

We will create a Vector Store on OpenAI API and upload our PDFs to the Vector Store. OpenAI will read those PDFs, separate the content into multiple chunks of text, run embeddings on those and store those embeddings and the text in the Vector Store. It will enable us to query this Vector Store to return relevant content based on a query.

## upload_single_pdf is a helper function designed to:

- Take a PDF file from your local machine, Opens the PDF in binary mode ('rb').

- Upload it to OpenAI’s file storage . Uploads the file to OpenAI using client.files.create.  

- The purpose="assistants" tag tells the system this file will be used in tools like vector stores or assistants.

- Attach that uploaded file to an existing vector store where it can later be searched using file search.


In [5]:
def upload_single_pdf(file_path: str, vector_store_id: str):
    file_name = os.path.basename(file_path)
    try:
        file_response = client.files.create(file=open(file_path, 'rb'), purpose="assistants") 
        attach_response = client.vector_stores.files.create(
            vector_store_id=vector_store_id,
            file_id=file_response.id  # the unique ID assigned to this uploaded file.
        )
        return {"file": file_name, "status": "success"}
    except Exception as e:
        print(f"Error with {file_name}: {str(e)}")
        return {"file": file_name, "status": "failed", "error": str(e)}

upload_pdf_files_to_vector_store  function:

-  Finds all PDFs in a folder
- Uploads them concurrently using 10 threads . Submits each PDF upload task to the thread pool
- Uses upload_single_pdf to handle each upload.
- Uses tqdm to show a progress bar while uploads complete.
- future.result() waits for the upload to finish and retrieves its return value.
- Tracks how many uploads succeed or fail
- Returns a complete upload report

In [6]:
def upload_pdf_files_to_vector_store(vector_store_id: str):
    pdf_files = [os.path.join(dir_pdfs, f) for f in os.listdir(dir_pdfs)]
    stats = {"total_files": len(pdf_files), "successful_uploads": 0, "failed_uploads": 0, "errors": []}
    
    print(f"{len(pdf_files)} PDF files to process. Uploading in parallel...")

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(upload_single_pdf, file_path, vector_store_id): file_path for file_path in pdf_files}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(pdf_files)):
            result = future.result()
            if result["status"] == "success":
                stats["successful_uploads"] += 1
            else:
                stats["failed_uploads"] += 1
                stats["errors"].append(result)

    return stats

## create_vector_store 

creates a new vector store using OpenAI’s API.


This function returns the basic metadata of the newly created store so you can reference it in later steps.

In [7]:
def create_vector_store(store_name: str) -> dict:
    try:
        vector_store = client.vector_stores.create(name=store_name)
        details = {
            "id": vector_store.id,
            "name": vector_store.name,
            "created_at": vector_store.created_at,
            "file_count": vector_store.file_counts.completed
        }
        print("Vector store created:", details)
        return details
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return {}

In [8]:
store_name = "openai_blog_store"
vector_store_details = create_vector_store(store_name)
upload_pdf_files_to_vector_store(vector_store_details["id"])

Vector store created: {'id': 'vs_692fedb450dc8191b5b5180d687cdc29', 'name': 'openai_blog_store', 'created_at': 1764748724, 'file_count': 0}
3 PDF files to process. Uploading in parallel...


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:17<00:00,  5.94s/it]


{'total_files': 3, 'successful_uploads': 3, 'failed_uploads': 0, 'errors': []}

# Standalone vector search

Now that our vector store is ready, we are able to query the Vector Store directly and retrieve relevant content for a specific query. Using the new [vector search API](https://platform.openai.com/docs/api-reference/vector-stores/search), we're able to find relevant items from our knowledge base without necessarily integrating it in an LLM query.

In [9]:
query = "What's Deep Research?"
search_results = client.vector_stores.search(
    vector_store_id=vector_store_details['id'],
    query=query
)

In [10]:
for result in search_results.data:
    print(str(len(result.content[0].text)) + ' of character of content from ' + result.filename + ' with a relevant score of ' + str(result.score))

3618 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.979164968744516
3601 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.9336562593621925
3584 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.9264829663142641
2646 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.9240999630209855
3194 of character of content from Deep research System Card _ OpenAI.pdf with a relevant score of 0.88356353384142
2969 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.8672050568661657
3644 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.8220330216618142
3068 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.7722818472064626
3350 of character of content from Introducing deep research _ OpenA

We can see that different size (and under-the-hood different texts) have been returned from the search query. They all have different relevancy score that are calculated by our ranker which uses hybrid search.



# Integrating search results with LLM in a single API call

However instead of querying the vector store and then passing the data into the Responses or Chat Completion API call, an even more convenient way to use this search results in an LLM query would be to plug use file_search tool as part of OpenAI Responses API.

In [11]:
query = "What's Deep Research?"
response = client.responses.create(
    input= query,
    model="gpt-4o-mini",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_details['id']],
    }]
)

# Extract annotations from the response
annotations = response.output[1].content[0].annotations
    
# Get top-k retrieved filenames
retrieved_files = set([result.filename for result in annotations])

print(f'Files used: {retrieved_files}')
print('Response:')
print(response.output[1].content[0].text) # 0 being the filesearch call

Files used: {'Deep research System Card _ OpenAI.pdf', 'Introducing deep research _ OpenAI.pdf'}
Response:
Deep Research is a new capability introduced by OpenAI that enables multi-step research tasks conducted through its ChatGPT platform. It allows users to conduct extensive research on complex topics, synthesizing information from various online sources efficiently and effectively. Here's a breakdown of what it entails:

1. **Functionality**: Deep Research works autonomously to find, analyze, and compile information across the internet, generating comprehensive reports akin to that of a research analyst. This capability is leveraged by an optimized version of OpenAI's upcoming model suited for web browsing and data analysis.

2. **User Applications**: It is designed for individuals engaged in intensive knowledge work—such as in finance, science, policy, and engineering—who require precise and reliable research. It can also assist consumers seeking personalized recommendations.

3. *

In [12]:
vector_store_details

{'id': 'vs_692fedb450dc8191b5b5180d687cdc29',
 'name': 'openai_blog_store',
 'created_at': 1764748724,
 'file_count': 0}

We can see that `gpt-4o-mini` was able to answer a query that required more recent, specialised knowledge about OpenAI's Deep Research. It used content from the file `Introducing deep research _ OpenAI.pdf` that had chunks of texts that were the most relevant. If we want to go even deeper in the analysis of chunk of text retrieved, we can also analyse the different texts that were returned by the search engine by adding `include=["output[*].file_search_call.search_results"]` to our query.


In [13]:
response

Response(id='resp_096ccb85c2b2014000692fedc9e8e481968cd3dcf139b4efd4', created_at=1764748745.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFileSearchToolCall(id='fs_096ccb85c2b2014000692fedcb41748196a0d9d79ac7189b67', queries=['What is Deep Research?'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_096ccb85c2b2014000692fedd1b56081969a749fcc23a2278b', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-XVAGedV4hUDf7VUnVWWCZ5', filename='Introducing deep research _ OpenAI.pdf', index=629, type='file_citation'), AnnotationFileCitation(file_id='file-XVAGedV4hUDf7VUnVWWCZ5', filename='Introducing deep research _ OpenAI.pdf', index=886, type='file_citation'), AnnotationFileCitation(file_id='file-XVAGedV4hUDf7VUnVWWCZ5', filename='Introducing deep research _ OpenAI.pdf', index=1076, type='file_citation'), AnnotationFileCitation(file

In [14]:
response = client.responses.create(
    input= query,
    model="gpt-4o-mini",
   
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_details['id']],
        
    }],
     include=["file_search_call.results"]
)

In [15]:
response

Response(id='resp_0c9ba4f2a93538a200692fede229848195b1e1ed062fbcdfb7', created_at=1764748770.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFileSearchToolCall(id='fs_0c9ba4f2a93538a200692fede33db88195ba090a74da9964f6', queries=['What is Deep Research?'], status='completed', type='file_search_call', results=[Result(attributes={}, file_id='file-XVAGedV4hUDf7VUnVWWCZ5', filename='Introducing deep research _ OpenAI.pdf', score=0.9846, text="February 2, 2025 Release\r\nIntroducing deep research\r\nAn agent that uses reasoning to synthesize large amounts of\r\nonline information and complete multi-step research tasks\r\nfor you.Available to Pro users today, Plus and Team next.\r\nTry on ChatGPT\r\n00:00 | 01:01\r\nListen to article 8:54 Share\r\n09/03/2025, 14:36 Introducing deep research | OpenAI\r\nhttps://openai.com/index/introducing-deep-research/ 1/36Today we’re launching deep research in ChatGPT

**Metadata filtering**

filter the search results based on the metadata of the files. For more details, refer to our retrieval guide, which covers:

-  set attributes on vector store files
-  define filters

```python

response = client.responses.create(
    model="gpt-4.1",
    input="What is deep research by OpenAI?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": ["<vector_store_id>"],
        "filters": {
            "type": "in",
            "key": "category",
            "value": ["blog", "announcement"]
        }
    }]
)
print(response) 
```

**Pricing:**

You will be charged based on the total storage used across all your vector stores, determined by the size of parsed chunks and their corresponding embeddings.

Storage	Cost

Up to 1 GB (across all stores)	Free

Beyond 1 GB	$0.10/GB/day

In [17]:
client.vector_stores.delete(
    vector_store_id="vs_692fedb450dc8191b5b5180d687cdc29"
)

VectorStoreDeleted(id='vs_692fedb450dc8191b5b5180d687cdc29', deleted=True, object='vector_store.deleted')

**Limitations**

- File and storage size constraints — each file must fit within certain size limits (e.g. at most ~ 512 MB in many setups).
  
- Scalability limits per vector store — while vector stores allow many files, there are often practical limits (and user reports) about hitting “vector store size limit reached” errors when storing large numbers of files. 

Retrieval limitations due to embedding-based similarity — embeddings capture semantic similarity, but they are not perfect: certain nuanced queries or exact-match needs (dates, code fragments, legal language, tables) may produce inaccurate or suboptimal results. Theoretically, embedding-based retrieval cannot guarantee perfect recall or correctness for all query-types. 
Graphlit

Limited retrieval flexibility and metadata filtering — custom metadata-based filtering is weak; there’s minimal deterministic control over filtering before search (e.g. by document type, author, date) beyond what’s already supported.